# Process Data for Equity Loss Analysis

## Data Sources
- Fulton County digest parcel data from 2011 to 2022 (selected for LUC=101, SFHs), excel
- Fulton County digest parcel data for 2022 (for geocoding), geojson
- Fulton County sales data from 2011 to 2022, txt
- Atlanta Neighborhood Statistical Areas with supplemental data from Census (), 2022, csv from Neighborhood Nexus
- Neighborhood characteristics? unknown

**Note: NSAs in DeKalb are excluded, we do not have data for all years**

Those neighborhoods are:
- Candler Park, Druid Hills
- Lake Claire
- East Lake
- Kirkwood
- Edgewood
- East Atlanta
- Emory University/Center for Disease Control
- Part of Morningside/Lenox Park

This leaves _ neighborhoods (see appendix for list)

In [12]:
import os
import csv
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', 150)
pd.options.display.float_format = '{:.2f}'.format

In [13]:
def contains_keyword(filename: str, keywords: list[str]) -> str:
    return any(keyword in filename for keyword in keywords)

def clean_and_cast_column(column: pd.Series, var_map: dict) -> pd.Series:
    to_dtype = var_map[column.name]
    fill_val = None
    
    if to_dtype == "int" or to_dtype == "float":
        fill_val = 0
    elif to_dtype == "string":
        fill_val = ""
    else:
        raise ValueError(f"{to_dtype} is not a valid data type!")
    
    if ((to_dtype == "int" or to_dtype == "float")
        and (column.dtype == "string" or column.dtype == "object")):
        # Remove commas from number strings before converting
        column = column.astype("str").str.replace(",", "").astype('float')
    
    # Record number of filled nulls
    print(f"Number of nulls in column {column.name}: {column.isna().sum()}")
    column = column.fillna(fill_val)
    return column.astype(to_dtype)

### Create a file with Fulton County digest data for all years

In [14]:
# Select desired digest files
FULTON_DIR = './data/raw_fulton/'
fulton_files = os.listdir(FULTON_DIR)

keywords = ["DIGEST", "NF", "SF"]
digest_cols = {
    "Taxyr": "int",
    "Parid": "string",
    "Situs Adrno": "int",
    "Situs Adrdir": "string",
    "Situs Adrstr": "string",
    "Situs Adrsuf": "string",
    "Cityname": "string",
    "Luc": "string",
    "Calcacres": "float",
    "Own1": "string",
    "Own2": "string",
    "Owner Adrno": "int",
    "Owner Adradd": "string",
    "Owner Adrdir": "string",
    "Owner Adrstr": "string",
    "Owner Adrsuf": "string",
    "Cityname.1": "string",
    "Statecode": "string",
    "Zip1": "string",
    "Aprtot": "float",
    "D Yrblt": "int",
    "D Effyr": "int",
    "D Yrremod": "int",
    "Sfla": "float"
}

desired_files = filter(lambda file: contains_keyword(file, keywords), fulton_files)

# Read desired files and only parse desired cols
# Need to ensure Luc is read in as a str so we can filter appropriately
desired_files_dfs = [
    pd.read_excel(
        FULTON_DIR + file,
        usecols=digest_cols,
        dtype={"Luc": "str"}
    ) for file in desired_files
]

In [15]:
# Concat selected digest files, select for LUC = 101 (SFH), drop complete duplicates
# Record total number of parcels
digest_full = pd.concat(desired_files_dfs)
digest_full = digest_full[digest_full['Luc'] == '101']

In [16]:
# After filtering for Luc, we can continue to cast other columns
rename = {
    "Taxyr": "TAXYR",
    "Parid": "PARID",
    "Cityname.1": "own_cityname",
    "Zip1": "own_zip"
}
init_len = len(digest_full)

digest_full = digest_full.drop_duplicates()
print(f"Init len: {init_len}. Number of dropped duplicates: {init_len - len(digest_full)}. Final len: {len(digest_full)}")

# Records nulls and set datatypes
for column in digest_full.columns:
    digest_full[column] = clean_and_cast_column(digest_full[column], digest_cols)
    
digest_full = digest_full.rename(columns=rename)

Init len: 3681749. Number of dropped duplicates: 896302. Final len: 2785447
Number of nulls in column Taxyr: 0
Number of nulls in column Parid: 0
Number of nulls in column Situs Adrno: 45
Number of nulls in column Situs Adrdir: 2783381
Number of nulls in column Situs Adrstr: 22
Number of nulls in column Situs Adrsuf: 189813
Number of nulls in column Cityname: 4973
Number of nulls in column Luc: 0
Number of nulls in column Calcacres: 572
Number of nulls in column Own1: 0
Number of nulls in column Own2: 2331350
Number of nulls in column Owner Adrno: 67785
Number of nulls in column Owner Adradd: 2780418
Number of nulls in column Owner Adrdir: 2723376
Number of nulls in column Owner Adrstr: 3572
Number of nulls in column Owner Adrsuf: 213850
Number of nulls in column Cityname.1: 3167
Number of nulls in column Statecode: 3482
Number of nulls in column Zip1: 4131
Number of nulls in column Aprtot: 0
Number of nulls in column D Yrblt: 3899
Number of nulls in column D Effyr: 2262400
Number of n

In [17]:
digest_full[digest_full['Owner Adrno'] == 0].sample(3)

,TAXYR,PARID,Situs Adrno,Situs Adrdir,Situs Adrstr,Situs Adrsuf,Cityname,Luc,Calcacres,Own1,Own2,Owner Adrno,Owner Adradd,Owner Adrdir,Owner Adrstr,Owner Adrsuf,own_cityname,Statecode,own_zip,Aprtot,D Yrblt,D Effyr,D Yrremod,Sfla
61038,2015,14F007900030370,1410,,MARTINIQUE,CT,FUL,101,0.58,FERNANDES RITA E,,0,,,P.O. BOX 311609,,ATLANTA,GA,31131,91700.00,1991,0,0,1553.00
34601,2014,14 008600060473,553,,MIDDLE,ST,ATLANTA,101,0.06,SHAODONG LIU,,0,,,,,,,,40000.00,2004,0,0,1320.00
83901,2013,14 010700060593,826,,WHITE,ST,ATLANTA,101,0.17,PRECISION INVESTMENT PROPERTIES LLC,,0,,,P.O. BOX 366512,,ATLANTA,GA,30336,25000.00,2005,0,0,1830.00


Note: cases where Owner Adrno is empty is often because it is a PO BOX in the Owner Adrstr column

In [18]:
# Quickly validate no data quality issues by looking at number of parcels per year
print(digest_full.shape)
digest_full.groupby("TAXYR")['PARID'].count()

(2785447, 24)


TAXYR
2010    208196
2011    208421
2012    208964
2013    209749
2014    210860
2015    212335
2016    202821
2017    216088
2018    217793
2019    219790
2020    222101
2021    223659
2022    224670
Name: PARID, dtype: int64

Note: It looks like a few parcels are lost between 2015 and 2016 (approx 10K) - this is potentially a flaw in Fulton data, not our own processing

In [19]:
# Export to CSV, Parquet
OUTPUT_PATH = 'output/fulton_parcels_all'
digest_full.to_csv(OUTPUT_PATH + '.csv', index=False)
digest_full.to_parquet(OUTPUT_PATH + '.parquet')

### Create a file with Fulton County sales data for all years

In [20]:
# Select desired sales files
keywords = ["STANDARDS SALES"]
sale_cols = {
    "Taxyr": "int",
    "Parid": "string",
    "Saledt": "string",
    "Luc": "string",
    "SALES PRICE": "float",
    "FAIR MARKET VALUE": "float",
    "DEED TYPE": "string",
    "Saleval": "string",
    "Costval": "string",
    "GRANTOR": "string",
    "GRANTEE": "string",
}

desired_files = filter(lambda file: contains_keyword(file, keywords), fulton_files)

desired_files_dfs = [
    pd.read_csv(
        FULTON_DIR + file,
        sep='\t',
        encoding='latin-1',
        usecols=sale_cols,
        quoting=csv.QUOTE_NONE,
        skipfooter=1,
        on_bad_lines="warn"
    ) for file in desired_files
]

C:\Users\Nick\AppData\Local\Temp\ipykernel_6400\1391998345.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pd.read_csv(
C:\Users\Nick\AppData\Local\Temp\ipykernel_6400\1391998345.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pd.read_csv(
C:\Users\Nick\AppData\Local\Temp\ipykernel_6400\1391998345.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pd.read_csv(
C:\Users\Nick\AppData\Local\Temp\ipykernel_6400\1391998345.py:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  pd.read_csv(
C:\Users\Nick\AppData\Local\Temp

In [21]:
# Concat selected digest files, select for LUC = 101 (SFH), drop complete duplicates
# Record total number of sales
rename = {
    "Taxyr": "TAXYR",
    "Parid": "PARID",
}

sales_full = pd.concat(desired_files_dfs)
sales_full = sales_full[sales_full['Luc'] == '101']

init_len = len(sales_full)
sales_full = sales_full.drop_duplicates()
print(f"Init len: {init_len}. Number of dropped duplicates: {init_len - len(sales_full)}")

# Records nulls and set datatypes
for column in sales_full.columns:
    sales_full[column] = clean_and_cast_column(sales_full[column], sale_cols)
    
sales_full = sales_full.rename(columns=rename)

Init len: 275814. Number of dropped duplicates: 461
Number of nulls in column Taxyr: 0
Number of nulls in column Parid: 0
Number of nulls in column Luc: 0
Number of nulls in column Saledt: 0
Number of nulls in column SALES PRICE: 19
Number of nulls in column FAIR MARKET VALUE: 0
Number of nulls in column DEED TYPE: 2
Number of nulls in column Costval: 0
Number of nulls in column Saleval: 118
Number of nulls in column GRANTOR: 18
Number of nulls in column GRANTEE: 8


In [22]:
# Check distribution of data for quick validation
sales_full.groupby("TAXYR")['PARID'].count()

TAXYR
2011    26777
2012    22684
2013    26074
2014    11711
2015    11256
2016    14978
2017    13210
2018    28281
2019    29134
2020    29570
2021    28645
2022    33033
Name: PARID, dtype: int64

In [23]:
sales_full.groupby("TAXYR").describe()

SALES PRICE                                                          \
            count      mean        std  min       25%       50%       75%   
TAXYR                                                                       
2011     26777.00 162950.94  630129.89 0.00      0.00  56050.00 189900.00   
2012     22684.00 164550.67  424239.44 0.00      0.00  49500.00 200515.00   
2013     26074.00 166081.78  440832.36 0.00      0.00  47000.00 212000.00   
2014     11711.00 331640.54  351925.01 0.00  79000.00 255000.00 455000.00   
2015     11256.00 357182.81  392959.23 0.00 101425.00 275000.00 475000.00   
2016     14978.00 348266.13  401923.49 0.00  85000.00 258325.00 474500.00   
2017     13210.00 386344.64  386724.40 0.00 139000.00 294000.00 517500.00   
2018     28281.00 268587.37  931256.97 0.00      1.00 108000.00 360000.00   
2019     29134.00 522038.36 2227323.74 0.00      1.00 137000.00 379900.00   
2020     29570.00 303400.40 1085088.76 0.00      1.00 155000.00 379000.00   
2021     28645.00 363893.16 2274967.99 0.00      1.00 193000.00 435000.00   
2022     33033.00 530580.09 1934406.18 0.00     10.00 250000.00 537500.00   

                   FAIR MARKET VALUE                                        \
               max             count      mean       std     min       25%   
TAXYR                                                                        
2011   51718108.00          26777.00 210525.70 292278.99    0.00  42100.00   
2012   20500000.00          22684.00 241854.35 325947.29 1500.00  56600.00   
2013   17038094.00          26074.00 238072.86 323868.49    0.00  39500.00   
2014    6350000.00          11711.00 294653.60 320690.06  100.00  63900.00   
2015   13914719.00          11256.00 314046.58 324030.38 1390.00  74887.50   
2016   17455046.00          14978.00 346104.72 368631.02    0.00  91500.00   
2017    7200000.00          13210.00 306637.00 320307.43    0.00  75725.00   
2018  135635876.00          28281.00 307217.92 359431.09    0.00  75800.00   
2019   40120000.00          29134.00 328591.65 367677.75  100.00 107200.00   
2020   58800000.00          29570.00 364300.06 393939.77    0.00 133500.00   
2021  126940000.00          28645.00 422773.73 432387.60    0.00 173100.00   
2022  231058000.00          33033.00 478192.98 476555.17  800.00 205000.00   

                                       
            50%       75%         max  
TAXYR                                  
2011  115300.00 276200.00  8750000.00  
2012  142800.00 314800.00 13698600.00  
2013  142225.00 325900.00  6322800.00  
2014  226960.00 402150.00  6189700.00  
2015  244970.00 428770.00  6151600.00  
2016  257500.00 470000.00  6000000.00  
2017  225900.00 425000.00  6000000.00  
2018  198500.00 418000.00  7150000.00  
2019  207000.00 427975.00  7861300.00  
2020  243900.00 467500.00 11350000.00  
2021  303700.00 529900.00 15000000.00  
2022  342600.00 587500.00  9966300.00

In [24]:
# Export to CSV, Parquet
OUTPUT_PATH = 'output/fulton_sales_all'
sales_full.to_csv(OUTPUT_PATH + '.csv', index=False)
sales_full.to_parquet(OUTPUT_PATH + '.parquet')

### Geocode digest data and determine NSA of each parcel

In [25]:
# Join all years digest to 2022 Fulton County geocoded parcel boundaries on ParcelID; e.g. geocode parcels
# Record # parcels not matched (ones that changed over the years, so now have a different ParcelID)
geo_parcels = gpd.read_file("./data/fulton_parcels.geojson")
geo_parcels = geo_parcels[["ParcelID", "OBJECTID", "geometry"]]
geo_parcels = geo_parcels.rename(columns={"ParcelID": "PARID"})

# DF of parcels with associated geometry
digest_full_geo = geo_parcels.merge(digest_full, on="PARID", how="inner")
print(f"Number of parcels unable to be matched: {len(digest_full) - len(digest_full_geo)}")

Number of parcels unable to be matched: 6819


In [26]:
# Create a DF of ATL NSA statistics from 2021 5 year ACS estimates (aquired from Neighborhood Nexus), use later for analysis
# Spatially join parcel geodata to Atlanta NSAs to get the neighborhood of each parcel
atl_nsa_stats = pd.read_csv("./data/atl_nsa_stats.csv")
atl_nsa_stats = atl_nsa_stats.rename(columns={"Details": "neighborhood"})

geo_atl_nsa = gpd.read_file("./data/atl_nsa.geojson")
geo_atl_nsa = geo_atl_nsa.rename(columns={"NEIGHBORHO": "neighborhood"})
geo_atl_nsa = geo_atl_nsa[["neighborhood", "geometry"]]

print(f"Validate CRS: {digest_full_geo.crs == geo_atl_nsa.crs}")

# DF with ATL NSA geometries and ACS statistics
# geo_atl_nsa = geo_atl_nsa.merge(atl_nsa_stats, on="neighborhood", how="inner")

# DF with all Fulton parcels and their matched neighborhood (some may not have a match)
digest_full_geo_nbhd = digest_full_geo.sjoin(geo_atl_nsa, how='left', predicate='within').drop(columns="index_right")

Validate CRS: True


In [27]:
# Drop NSAs from Atlanta NSA df which has no matches (e.g. those outside of Fulton County) - tho we want to retain all Fulton for comparison
# Write the NSAs we include and those we don't to a text file for Appendix
digest_atl_geo_nbhd = digest_full_geo_nbhd[digest_full_geo_nbhd['neighborhood'].notna()]
print(f"Number of matched parcels in ATL: {len(digest_atl_geo_nbhd)}")

Number of matched parcels in ATL: 1009001


In [28]:
# Cast geometry column to string for Parquet
digest_full_geo_nbhd["geometry"] = digest_full_geo_nbhd["geometry"].astype("string")
digest_atl_geo_nbhd["geometry"] = digest_atl_geo_nbhd["geometry"].astype("string")

# Output intermediary data for analysis
OUTPUT_PATH = './output/'

digest_full_geo_nbhd.to_csv(OUTPUT_PATH + 'digest_full_geo_nbhd.csv', index=False)
digest_full_geo_nbhd.to_parquet(OUTPUT_PATH + 'digest_full_geo_nbhd.parquet')

digest_atl_geo_nbhd.to_csv(OUTPUT_PATH + 'digest_atl_geo_nbhd.csv', index=False)
digest_atl_geo_nbhd.to_parquet(OUTPUT_PATH + 'digest_atl_geo_nbhd.parquet')

C:\Users\Nick\AppData\Local\Temp\ipykernel_6400\556227967.py:2: UserWarning: Geometry column does not contain geometry.
  digest_full_geo_nbhd["geometry"] = digest_full_geo_nbhd["geometry"].astype("string")
C:\Users\Nick\AppData\Local\Temp\ipykernel_6400\556227967.py:3: UserWarning: Geometry column does not contain geometry.
  digest_atl_geo_nbhd["geometry"] = digest_atl_geo_nbhd["geometry"].astype("string")
c:\Users\Nick\Documents\code\equity-extraction\equity-extraction\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
